In [62]:
%pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [63]:
tickers = 'AAPL,TSLA,MSFT,'
indices = 'SPY,'

md = yf.download(
    tickers = indices + tickers,
    group_by='ticker',
    start='2020-05-19', end='2021-05-19',
    interval = '1d',
    auto_adjust = True,
    threads = True
    )

print(f'Downloaded {len(md)} days of market data for indices : {indices} and tickers : {tickers}')

md['SPY'].tail()
print(f"Index : {md.index[:5]}...")
print(f"Columns : {md.columns[:5]}...")



[*********************100%***********************]  4 of 4 completed
Downloaded 253 days of market data for indices : SPY, and tickers : AAPL,TSLA,MSFT,
Index : DatetimeIndex(['2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21',
               '2020-05-22'],
              dtype='datetime64[ns]', name='Date', freq=None)...
Columns : MultiIndex([('TSLA',   'Open'),
            ('TSLA',   'High'),
            ('TSLA',    'Low'),
            ('TSLA',  'Close'),
            ('TSLA', 'Volume')],
           )...


In [64]:
def getPriceAndReturn(md):
    priceField = 'Close'
    prices = pd.DataFrame()
    returns = pd.DataFrame()
    for ticker, kind in md.columns:
        if kind == priceField:
            prices[ticker] = md[ticker][[priceField]].values.reshape(1, -1).ravel()
            returns[ticker] = prices[ticker].pct_change()
    return prices, returns

prices, returns = getPriceAndReturn(md)

print(prices['SPY'].head())
print(returns['SPY'].head())



0    290.343842
1    287.361664
2    292.243408
3    290.225769
4    290.776947
Name: SPY, dtype: float64
0         NaN
1   -0.010271
2    0.016988
3   -0.006904
4    0.001899
Name: SPY, dtype: float64


In [65]:
def getMeanAndStd(returns):
    mean = {}
    std = {}

    for ticker in returns.columns:
        mean[ticker] = returns[ticker].mean()
        std[ticker] = returns[ticker].std()

    return mean, std

mean, std = getMeanAndStd(returns)

print(f"SPY mean : {mean['SPY']}")
print(f"SPY std  : {std['SPY']}")

SPY mean : 0.0014474968820360192
SPY std  : 0.010791323551446576


In [91]:
def getCovAndCorr(returns, std):
    cov = []
    corr = []

    for i in range(len(returns.columns)):
        covColumn = []
        corrColumn = []
        
        iname = returns.columns[i]
        for j in range(i):
            covColumn.append(cov[j][i])
            corrColumn.append(corr[j][i])
        iStd = std[iname]
        covColumn.append(iStd * iStd)
        corrColumn.append(1)
        for j in range(i+1, len(returns.columns)):
            jname = returns.columns[j]
            jStd = std[jname]
            tickerCov = returns[iname].cov(returns[jname])
            covColumn.append(tickerCov)
            corrColumn.append(tickerCov / (iStd * jStd))
        cov.append(covColumn)
        corr.append(corrColumn)

    covDf = pd.DataFrame(cov, index = returns.columns, columns = returns.columns)
    #for 
    corrDf = pd.DataFrame(corr, index = returns.columns, columns = returns.columns)
    return covDf, corrDf

cov, corr = getCovAndCorr(returns, std)
'''
print(f"Covariance :\n{cov}")
print(f"Test Covariance TSLA MSFT: {returns['TSLA'].cov(returns['MSFT'])}")
print(f"Correlation :\n{corr}")
print(f"Test Correlation TSLA MSFT: {returns['TSLA'].cov(returns['MSFT'])/(std['TSLA'] * std['MSFT'])}")
print(f"Covarience SPY column :\n{cov['SPY']}")
print(f"Covarience TSLA row :\n{cov.loc['TSLA']}")
print(f"corr.loc['SPY','TSLA'] : {corr.loc['SPY','TSLA']}")
'''
print(f"corr.loc['TSLA','SPY'] : {corr.loc['TSLA','SPY']}")


corr.loc['TSLA','SPY'] : 0.4642533884028333
